In [1]:
from __future__ import print_function # Python 2/3 compatibility
import os
import glob
import numpy as np
from PIL import Image
import tifffile
from tifffile import imwrite
import numpy
from skimage import io
import matplotlib.pyplot as plt
import pywt

from skimage import (
    data, restoration, util
)


In [2]:
def explore(image_data):
    
    print("Image shape:", image_data.shape)

    mean_value = np.mean(image_data)
    std_value = np.std(image_data)
    min_value = np.min(image_data)
    max_value = np.max(image_data)

    print("Mean:", mean_value)
    print("Standard Deviation:", std_value)
    print("Minimum:", min_value)
    print("Maximum:", max_value)
    
def convert_to_greyscale(image, weights):
    # Define the weights
    weights = np.array(weights)
    
    # Calculate the weighted sum along the last axis (color channels)
    greyscale_image = np.dot(image, weights)
    
    # Optional: Convert to desired dtype (if not 'float')
    greyscale_image = greyscale_image.astype(np.uint8)
    
    return greyscale_image

def get_proportions(arg_list):
    """
    Get adjusted weights for grayscale conversion based on presence/absence of RGB channels.

    Parameters:
    - r, g, b: Binary values indicating presence (1) or absence (0) of the R, G, and B channels.

    Returns:
    - (Wr, Wg, Wb): Tuple containing the adjusted weights for R, G, and B channels.
    """
    r = arg_list[0]
    g = arg_list[1]
    b = arg_list[2]
    
    # Original weights
    #Wr_original, Wg_original, Wb_original = 0.299, 0.587, 0.114
    Wr_original, Wg_original, Wb_original = 0.333, 0.333, 0.333
    
    # If all channels are absent, return (0, 0, 0) as special case
    if r == g == b == 0:
        return [0, 0, 0]

    # Calculating adjusted weights while considering original proportions
    total_original_weight = r * Wr_original + g * Wg_original + b * Wb_original
    
    Wr = round((Wr_original * r) / total_original_weight, 3) if total_original_weight else 0
    Wg = round((Wg_original * g) / total_original_weight, 3) if total_original_weight else 0
    Wb = round((Wb_original * b) / total_original_weight, 3) if total_original_weight else 0

    return [Wr, Wg, Wb]

def plot_result(image, background):
    fig, ax = plt.subplots(nrows=1, ncols=3)

    ax[0].imshow(image, cmap='gray')
    ax[0].set_title('Original image')
    ax[0].axis('off')

    ax[1].imshow(background, cmap='gray')
    ax[1].set_title('Background')
    ax[1].axis('off')

    ax[2].imshow(image - background, cmap='gray')
    ax[2].set_title('Result')
    ax[2].axis('off')

    fig.tight_layout()

def get_scaled_prop(max_values):
    """
    Calculates the proportion of each channel based on the provided max_values and rounds it to 3 decimal places.
    
    Parameters:
        max_values (list or tuple): A list or tuple containing the maximum values for each channel.
        
    Returns:
        list: A list containing the proportion of each channel, rounded to 3 decimal places.
    """
    # Ensure max_values is a list or tuple and has exactly 3 elements for the RGB channels.
    if not (isinstance(max_values, (list, tuple)) and len(max_values) == 3):
        raise ValueError("max_values must be a list or tuple with exactly 3 elements.")
    
    # Calculate the sum of max_values to get the normalization factor.
    total = sum(max_values)
    
    # Avoid division by zero by returning equal proportions if total is 0.
    if total == 0:
        return [1/3, 1/3, 1/3]
    
    # Calculate the proportion of each channel, round to 3 decimal places and return the results.
    proportions = [round(val / total, 3) for val in max_values]
    return proportions

#takes input a 3D image with RGB channels
#return a list of length 3 with the maximum value for each input channel
#it can be easily changed to mean if required
def getmaxes(marker):
    maxes = []
    for i in range(3):
        cr_channel = marker[:, :, i]
        maxes.append(np.max(cr_channel))
    
    return maxes


In [5]:
#path = "/home/carol/NaroNet-main/NaroNet-main/Endometrial_POLE/Raw_Data/Images/Endometrial_ca_17_15524_[10021,45166]_component_data.tif"
path2 = "/home/carol/NaroNet-main/Datasets/Images_noconv_noroll/h17_6226.tif"

# img = Image.open(path)
# print(np.shape(img))
#endo = io.imread(path)
lcan = io.imread(path2)

# greysc = convert_to_greyscale(lcan)
# coins_image = data.coins()
# print(greysc[0].dtype)
# print(coins_image.dtype)

In [ ]:
#exploring RGb image
path2 = "/home/carol/NaroNet-main/Datsets/Images_noconv_noroll/h17_6226.tif"
lcan = io.imread(path2)
for marker in range(6):
    print("Exploring marker "+str(marker)+"\n")
    cr_marker = lcan[marker]
    for channel in range(3):
        cr_channel = cr_marker[:, :, channel]
        print("Exploring channel "+str(channel)+"\n")
        explore(cr_channel)

In [ ]:
#converting a whole folder D2
current_directory = "/home/carol/NaroNet-main/Datasets/Images_noconv_noroll"
name = 0
# Iterate through all files in the directory
for filename in os.listdir(current_directory):
    # Create a full file path
    cr_img = io.imread(current_directory+'/'+filename)
    gray_markers = []
    for marker in range(6):
        cr_marker = cr_img[marker]
        maxes = getmaxes(cr_marker)
        prop = get_scaled_prop(maxes)
        gray_marker = convert_to_greyscale(cr_marker, prop)
        # roll_ball = gray_marker - restoration.rolling_ball(gray_marker)
        
        # Append filtered and converted image to our list
        gray_markers.append(gray_marker)

    # Stack all grayscale markers into a single array along a new first axis
    final_image = np.stack(gray_markers, axis=0)

    str_name = str(name)+'.tif'
    imwrite(str_name, final_image)
    print("Generated "+str_name+" for "+filename)
    
    name = name+1

In [ ]:
#converting a whole folder D1
current_directory = "/home/carol/NaroNet-main/Datasets/Images_noconv_noroll"
name = 0
# Iterate through all files in the directory
for filename in os.listdir(current_directory):
    print(filename)
    # Create a full file path
    cr_img = io.imread(current_directory+'/'+filename)
    gray_markers = []
    for marker in range(6):
        print(marker)
        cr_marker = cr_img[marker]
        missing_channels = [1,1,1]
        for channel in range(3):
            #cr_channel = cr_marker[:, :, channel]
            print(cr_marker.shape)
            if np.max(cr_channel) == 0:
                missing_channels[channel] = 0

        prop = get_proportions(missing_channels)
        gray_marker = convert_to_greyscale(cr_marker, prop)
        roll_ball = gray_marker - restoration.rolling_ball(gray_marker)
        
        # Append filtered and converted image to our list
        gray_markers.append(roll_ball)

    # Stack all grayscale markers into a single array along a new first axis
    final_image = np.stack(gray_markers, axis=0)

    str_name = str(name)+'.tif'
    imwrite(str_name, final_image)
    print("Generated "+str_name+" for "+filename)
    
    name = name+1

In [6]:
import cv2
from cv2_rolling_ball import subtract_background_rolling_ball

In [8]:
path = "/home/carol/NaroNet-main/D2/h17_6226.tif"
lcan = io.imread(path)
img, background = subtract_background_rolling_ball(lcan[0], 30, light_background=False,
                                     use_paraboloid=False, do_presmooth=True)

In [11]:
print(background.shape)
print(lcan[0].shape)

(26312, 5138)
(26312, 5138)


In [3]:
path1 = "/home/carol/NaroNet-main/Datasets/D2/h16_11096.tif"
er1 = io.imread(path1)

path2 = "/home/carol/NaroNet-main/Datasets/D2/h16_19144.tif"
er2 = io.imread(path2)

print(er1.shape)
print(er2.shape)

(6, 15080, 22626)
(6, 45033, 15131)


In [5]:
explore(er2[1])

Image shape: (45033, 15131)
Mean: 4.449137606037848
Standard Deviation: 11.890435782905497
Minimum: 0
Maximum: 254


In [3]:
#converting a whole folder to D3 - greyscale to gs+ roll ball
#GPU cannot be used for rolling_ball algorithm -> the most expensive operation
current_directory = "/home/carol/NaroNet-main/Datasets/D2"
name = 1
# Iterate through all files in the directory
for filename in os.listdir(current_directory):
    cr_img = io.imread(current_directory+'/'+filename)
    if filename == "h17_6226.tif" or filename == "h16_16787.tif":
        continue
    gray_markers = []
    for marker in range(6):
        cr_marker = cr_img[marker]
        roll_ball = cr_marker - restoration.rolling_ball(cr_marker)
        
        # Append filtered and converted image to our list
        gray_markers.append(roll_ball)

    # Stack all grayscale markers into a single array along a new first axis
    final_image = np.stack(gray_markers, axis=0)

    str_name = str(name)+'.tif'
    imwrite(str_name, final_image)
    print("Generated "+str_name+" for "+filename)
    
    name = name+1

Generated 1.tif for h16_19144.tif
Generated 2.tif for h16_28421.tif
Generated 3.tif for h13_10873.tif
Generated 4.tif for h17_6794.tif
Generated 5.tif for h16_9373.tif
Generated 6.tif for h14_28759.tif
Generated 7.tif for h16_11096.tif
Generated 8.tif for h16_15119.tif
Generated 9.tif for h16_12101.tif
Generated 10.tif for h16_15767.tif
Generated 11.tif for h16_7700.tif


In [6]:
#converting a single image - D2
gray_markers = []
for marker in range(6):
    cr_marker = lcan[marker]
    missing_channels = [1,1,1]

    #present channel, no max/mean
    # for channel in range(3):
    #     cr_channel = cr_marker[:, :, channel]
    #     if np.max(cr_channel) == 0:
    #         missing_channels[channel] = 0

    maxes = getmaxes(cr_marker)
    prop = get_scaled_prop(maxes)
    gray_marker = convert_to_greyscale(cr_marker, prop)
    #roll_ball = gray_marker - restoration.rolling_ball(gray_marker)
    
    # Append the gray_marker to our list
    gray_markers.append(gray_marker)

# Stack all grayscale markers into a single array along a new first axis
final_image = np.stack(gray_markers, axis=0)
imwrite("grey_maxes_h17_6226.tif", final_image)
